# Tidy Tuesday practice

Practice wrangling - especially for people-centred data.

[Astronaut data sauce](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-07-14/readme.md)

Can find all tidy data sources [here](https://github.com/rfordatascience/tidytuesday)

In [1]:
import os
import tarfile
from six.moves import urllib
import pandas as pd

## Get data

In [17]:
def fetch_astronaut_data(data_url="", data_path=""):
    if not os.path.isfile(data_path):
        urllib.request.urlretrieve(data_url, data_path)

In [164]:
astronaut_url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-07-14/astronauts.csv'
astronaut_path = './data/astronauts.csv'

fetch_astronaut_data(astronaut_url, astronaut_path)
astronaut = pd.read_csv(astronaut_path)

In [22]:
astronaut.head()

,id,number,nationwide_number,name,original_name,sex,year_of_birth,nationality,military_civilian,selection,...,year_of_mission,mission_title,ascend_shuttle,in_orbit,descend_shuttle,hours_mission,total_hrs_sum,field21,eva_hrs_mission,total_eva_hrs
0,1,1,1,"Gagarin, Yuri",ГАГАРИН Юрий Алексеевич,male,1934,U.S.S.R/Russia,military,TsPK-1,...,1961,Vostok 1,Vostok 1,Vostok 2,Vostok 3,1.77,1.77,0,0.0,0.0
1,2,2,2,"Titov, Gherman",ТИТОВ Герман Степанович,male,1935,U.S.S.R/Russia,military,TsPK-1,...,1961,Vostok 2,Vostok 2,Vostok 2,Vostok 2,25.00,25.30,0,0.0,0.0
2,3,3,1,"Glenn, John H., Jr.","Glenn, John H., Jr.",male,1921,U.S.,military,NASA Astronaut Group 1,...,1962,MA-6,MA-6,MA-6,MA-6,5.00,218.00,0,0.0,0.0
3,4,3,1,"Glenn, John H., Jr.","Glenn, John H., Jr.",male,1921,U.S.,military,NASA Astronaut Group 2,...,1998,STS-95,STS-95,STS-95,STS-95,213.00,218.00,0,0.0,0.0
4,5,4,2,"Carpenter, M. Scott","Carpenter, M. Scott",male,1925,U.S.,military,NASA- 1,...,1962,Mercury-Atlas 7,Mercury-Atlas 7,Mercury-Atlas 7,Mercury-Atlas 7,5.00,5.00,0,0.0,0.0


In [23]:
astronaut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277 entries, 0 to 1276
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        1277 non-null   int64  
 1   number                    1277 non-null   int64  
 2   nationwide_number         1277 non-null   int64  
 3   name                      1277 non-null   object 
 4   original_name             1272 non-null   object 
 5   sex                       1277 non-null   object 
 6   year_of_birth             1277 non-null   int64  
 7   nationality               1277 non-null   object 
 8   military_civilian         1277 non-null   object 
 9   selection                 1272 non-null   object 
 10  year_of_selection         1277 non-null   int64  
 11  mission_number            1277 non-null   int64  
 12  total_number_of_missions  1277 non-null   int64  
 13  occupation                1277 non-null   object 
 14  year_of_

## Data wrangle

Start by splitting the name into first name and last name.

In [40]:
astronaut['name']

0                 Gagarin, Yuri
1                Titov, Gherman
2           Glenn, John H., Jr.
3           Glenn, John H., Jr.
4           Carpenter, M. Scott
                 ...           
1272    McClain, Anne Charlotte
1273            Koch, Christina
1274             Morgan, Andrew
1275              Meir, Jessica
1276        Al Mansoori, Hazzaa
Name: name, Length: 1277, dtype: object

In [146]:
def clean_astronaut_name(astronaut_name):
    last_name = astronaut_name.str.split(pat=',', expand=True)[0]
    first_name = astronaut_name.str.split(pat=',', expand=True)[1]
    first_name = first_name.str.replace("[A-Z]\.", "")
    first_name = first_name.str.strip()
    clean_name = pd.DataFrame({
        'first_name': first_name,
        'last_name': last_name
    })
    return(clean_name)

# Subset the astronauts' names into own dataset so I can clean separately
astronaut_name_series = astronaut['name']
astronaut_clean_name = clean_astronaut_name(astronaut_name_series)

In [165]:
# Get current columns and append the new names to reorder after concat
col_order = astronaut.columns.tolist()
col_order = col_order[0:3] + ['first_name', 'last_name'] + col_order[3:]

# Append to dataframe
astronaut = pd.concat([astronaut_clean_name, astronaut], axis=1)

# Rearrange columns
astronaut = astronaut[col_order]

astronaut.head()

,id,number,nationwide_number,first_name,last_name,name,original_name,sex,year_of_birth,nationality,...,year_of_mission,mission_title,ascend_shuttle,in_orbit,descend_shuttle,hours_mission,total_hrs_sum,field21,eva_hrs_mission,total_eva_hrs
0,1,1,1,Yuri,Gagarin,"Gagarin, Yuri",ГАГАРИН Юрий Алексеевич,male,1934,U.S.S.R/Russia,...,1961,Vostok 1,Vostok 1,Vostok 2,Vostok 3,1.77,1.77,0,0.0,0.0
1,2,2,2,Gherman,Titov,"Titov, Gherman",ТИТОВ Герман Степанович,male,1935,U.S.S.R/Russia,...,1961,Vostok 2,Vostok 2,Vostok 2,Vostok 2,25.00,25.30,0,0.0,0.0
2,3,3,1,John,Glenn,"Glenn, John H., Jr.","Glenn, John H., Jr.",male,1921,U.S.,...,1962,MA-6,MA-6,MA-6,MA-6,5.00,218.00,0,0.0,0.0
3,4,3,1,John,Glenn,"Glenn, John H., Jr.","Glenn, John H., Jr.",male,1921,U.S.,...,1998,STS-95,STS-95,STS-95,STS-95,213.00,218.00,0,0.0,0.0
4,5,4,2,Scott,Carpenter,"Carpenter, M. Scott","Carpenter, M. Scott",male,1925,U.S.,...,1962,Mercury-Atlas 7,Mercury-Atlas 7,Mercury-Atlas 7,Mercury-Atlas 7,5.00,5.00,0,0.0,0.0


Add age of mission

In [168]:
astronaut['mission_age'] = astronaut.year_of_mission - astronaut.year_of_birth

astronaut.head()

,id,number,nationwide_number,first_name,last_name,name,original_name,sex,year_of_birth,nationality,...,mission_title,ascend_shuttle,in_orbit,descend_shuttle,hours_mission,total_hrs_sum,field21,eva_hrs_mission,total_eva_hrs,mission_age
0,1,1,1,Yuri,Gagarin,"Gagarin, Yuri",ГАГАРИН Юрий Алексеевич,male,1934,U.S.S.R/Russia,...,Vostok 1,Vostok 1,Vostok 2,Vostok 3,1.77,1.77,0,0.0,0.0,27
1,2,2,2,Gherman,Titov,"Titov, Gherman",ТИТОВ Герман Степанович,male,1935,U.S.S.R/Russia,...,Vostok 2,Vostok 2,Vostok 2,Vostok 2,25.00,25.30,0,0.0,0.0,26
2,3,3,1,John,Glenn,"Glenn, John H., Jr.","Glenn, John H., Jr.",male,1921,U.S.,...,MA-6,MA-6,MA-6,MA-6,5.00,218.00,0,0.0,0.0,41
3,4,3,1,John,Glenn,"Glenn, John H., Jr.","Glenn, John H., Jr.",male,1921,U.S.,...,STS-95,STS-95,STS-95,STS-95,213.00,218.00,0,0.0,0.0,77
4,5,4,2,Scott,Carpenter,"Carpenter, M. Scott","Carpenter, M. Scott",male,1925,U.S.,...,Mercury-Atlas 7,Mercury-Atlas 7,Mercury-Atlas 7,Mercury-Atlas 7,5.00,5.00,0,0.0,0.0,37


## Explore stats

Check to make sure number is unique to name (and vice versa).

In [150]:
grouped_name_num = astronaut.groupby(['number', 'name'])['number', 'name'].nunique().add_prefix('n_').reset_index()
grouped_name_num[grouped_name_num.n_number != 1]

C:\Users\Koo\.conda\envs\naive_bayes\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,number,name,n_number,n_name


### Calculate age of first mission

Ties are resolved using `.first()` or `.last()`

Also `.rename()` doesn't change the df in place. Need to assign to variable.

In [219]:
age_first_mission = astronaut.groupby('name')['mission_age'].nsmallest(1).groupby(level='name').first().reset_index()

age_first_mission = age_first_mission.rename(columns={'mission_age': 'first_mission_age'})

age_first_mission.head()

,name,first_mission_age
0,"Acaba, Joseph M.",42
1,"Acton, Loren Wilbur",42
2,"Adamson, James C.",43
3,"Afanasyev, Viktor Mikhaylovich",43
4,Aidyn (Aydyn) Akanovich Aimbetov,43


In [200]:
age_first_mission.describe()

,first_mission_age
count,564.000000
mean,40.310284
std,5.262336
min,26.000000
25%,37.000000
50%,40.000000
75%,43.000000
max,61.000000


Mean age of first mission around 40.3 years. Youngest astronaut 26 years. Oldest age of first mission is 61 years.

In [220]:
# Left join sex from original dataframe
age_first_mission = age_first_mission.join(astronaut[['name', 'sex']].drop_duplicates().set_index('name'), on='name')
age_first_mission.head()

,name,first_mission_age,sex
0,"Acaba, Joseph M.",42,male
1,"Acton, Loren Wilbur",42,male
2,"Adamson, James C.",43,male
3,"Afanasyev, Viktor Mikhaylovich",43,male
4,Aidyn (Aydyn) Akanovich Aimbetov,43,male


The `.assign()` method allows me to assign new column. [Sauce](https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/) *Note: the `.insert()` method didn't seem to work - could it be because the object doesn't exist so it can't be modified in place?*

In [266]:
age_first_mission.groupby('sex').mean()

# Version with group sizes added
age_first_mission.groupby('sex').mean().assign(n_size = age_first_mission.groupby('sex').size().values)

,first_mission_age,n_size
sex,,
female,38.125,64
male,40.590,500


Average age of females slightly younger than males. There are a lot more male astronauts.

### Number of times on a mission

In [198]:
astronaut['name'].value_counts()

Ross, Jerry L.             7
Chang-Diaz, Franklin R.    7
Malenchenko, Yuri          6
Krikalev, Sergei           6
Foale, C. Michael          6
                          ..
Furrer, Reinhard Alfred    1
Brown, David M.            1
Bondar, Roberta Lynn       1
Wang, Taylor Gun-Jin       1
Treshchov, Sergei          1
Name: name, Length: 564, dtype: int64